# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

The primary business objective is to identify the key factors that drive the prices of used cars. This understanding will enable a used car dealership to make informed decisions about purchasing and pricing their inventory.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [1]:
import pandas as pd
#loading the dataset for the Data Understanding phase
df = pd.read_csv('Downloads/vehicles.csv')

# Data Understanding Steps

# 1. Initial Data Examination
initial_examination = {
    'First Five Rows': df.head(),
    'Dataset Shape': df.shape,
    'Feature Names': df.columns.tolist()
}

# 2. Statistical Summary
statistical_summary = df.describe(include='all')

# 3. Data Quality Assessment
missing_values = df.isnull().sum()
data_types = df.dtypes

# Compiling the results
data_understanding_results = {
    'Initial Examination': initial_examination,
    'Statistical Summary': statistical_summary,
    'Missing Values': missing_values,
    'Data Types': data_types
}

data_understanding_results



{'Initial Examination': {'First Five Rows':            id                  region  price  year manufacturer model  \
  0  7222695916                prescott   6000   NaN          NaN   NaN   
  1  7218891961            fayetteville  11900   NaN          NaN   NaN   
  2  7221797935            florida keys  21000   NaN          NaN   NaN   
  3  7222270760  worcester / central MA   1500   NaN          NaN   NaN   
  4  7210384030              greensboro   4900   NaN          NaN   NaN   
  
    condition cylinders fuel  odometer title_status transmission  VIN drive  \
  0       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
  1       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
  2       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
  3       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
  4       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
  
    size type paint_color st

# Data Understanding: Summary of Findings

Initial Data Examination:
Dataset Shape: The dataset contains 426,880 rows and 18 columns.
Feature Names: The dataset includes features like 'region', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color', and 'state'.
First Five Rows: The initial rows show a mix of numerical and categorical data, with several missing values in columns like 'year', 'manufacturer', 'model', etc.
    
    
Statistical Summary:    
The descriptive statistics provide an overview of numerical features like 'price' and 'year', as well as categorical features such as 'manufacturer' and 'model'.
Key observations include a wide range of car prices and years, with certain manufacturers being more prevalent in the dataset.

Data Quality Assessment:
Missing Values: There are notable missing values in several columns, particularly 'condition', 'cylinders', 'manufacturer', 'model', 'VIN', 'drive', 'size', 'type', and 'paint_color'.
Data Types: The data types are generally appropriate for the nature of the data in each column, with numerical features such as 'price' and 'year', and categorical features like 'manufacturer' and 'condition'.


### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# Data Preparation Steps

# 1. Data Cleaning

# Handling missing values by dropping rows where crucial information is missing
df_clean = df.dropna(subset=['year', 'manufacturer', 'model', 'odometer', 'fuel', 'transmission'])

# Handling outliers - focusing on the 'price' and 'odometer' columns
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

df_clean = remove_outliers(df_clean, 'price')
df_clean = remove_outliers(df_clean, 'odometer')

# 2. Feature Engineering

# Creating a new feature 'car_age'
current_year = pd.to_datetime('now').year
df_clean['car_age'] = current_year - df_clean['year']

# 3. Data Transformations

# Logarithmic Transformation of 'odometer'
df_clean['odometer_log'] = np.log(df_clean['odometer'] + 1)

# One-hot encoding of categorical variables
categorical_features = ['manufacturer', 'fuel', 'transmission']
df_clean = pd.get_dummies(df_clean, columns=categorical_features, drop_first=True)

# 4. Data Splitting

# Defining features and target variable
features = df_clean.drop(['price', 'id', 'region', 'model', 'condition', 'cylinders', 'title_status', 'VIN', 'drive', 'size', 'type', 'paint_color', 'state', 'year', 'odometer'], axis=1)
target = df_clean['price']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Displaying the shape of the training and testing sets
data_split_shapes = {
    'Training Features Shape': X_train.shape,
    'Testing Features Shape': X_test.shape,
    'Training Labels Shape': y_train.shape,
    'Testing Labels Shape': y_test.shape
}

data_split_shapes



/var/folders/s_/v1gz0yqs2z9bbg18b7h118880000gn/T/ipykernel_85470/2709023453.py:30: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  current_year = pd.to_datetime('now').year


{'Training Features Shape': (309020, 48),
 'Testing Features Shape': (77256, 48),
 'Training Labels Shape': (309020,),
 'Testing Labels Shape': (77256,)}

# Data Preparation Summary

The dataset has been prepared for modeling with the following steps:

Data Cleaning:
Rows with missing values in key columns ('year', 'manufacturer', 'model', 'odometer', 'fuel', 'transmission') were dropped.
Outliers in the 'price' and 'odometer' columns were removed using the Interquartile Range (IQR) method.


Feature Engineering:
A new feature 'car_age' was created by subtracting the car's year from the current year.
This feature provides a direct measure of the car's age, which can be a significant predictor of price.

Data Transformations:
A logarithmic transformation was applied to the 'odometer' column to reduce skewness and normalize the distribution.
Categorical variables ('manufacturer', 'fuel', 'transmission') were one-hot encoded, converting them into a format suitable for machine learning models.

Data Splitting:
The dataset was split into training and testing sets, with 80% of the data used for training and 20% for testing.
The training set contains 309,020 samples, and the testing set contains 77,256 samples.


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Building Linear Regression Model as a baseline
linear_model = LinearRegression()
linear_cv_scores = cross_val_score(linear_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Mean and standard deviation of the MSE scores (Linear Regression)
linear_cv_mse_mean = -linear_cv_scores.mean()
linear_cv_mse_std = linear_cv_scores.std()

# Building Random Forest Model
random_forest_model = RandomForestRegressor(n_estimators=50, min_samples_split=50, random_state=42)
rf_cv_scores = cross_val_score(random_forest_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Mean and standard deviation of the MSE scores (Random Forest)
rf_cv_mse_mean = -rf_cv_scores.mean()
rf_cv_mse_std = rf_cv_scores.std()

# Compiling the results
modeling_results = {
    'Linear Regression MSE Mean': linear_cv_mse_mean,
    'Linear Regression MSE Std': linear_cv_mse_std,
    'Random Forest MSE Mean': rf_cv_mse_mean,
    'Random Forest MSE Std': rf_cv_mse_std
}

modeling_results



{'Linear Regression MSE Mean': 116765221.0790455,
 'Linear Regression MSE Std': 781666.1754507449,
 'Random Forest MSE Mean': 64458432.962604426,
 'Random Forest MSE Std': 572737.4606349036}

# Analysis of Model Results

Linear Regression Results:
The Linear Regression model was evaluated using 5-fold cross-validation.
The mean and standard deviation of the Mean Squared Error (MSE) provide a measure of the model's average performance and variability across different folds.


Linear Regression:
MSE Mean: Approximately 116,765,221
MSE Standard Deviation: Approximately 781,666
These values suggest that the Linear Regression model has a relatively high error in predicting car prices, with some variability in its performance across different folds of cross-validation.

Random Forest:
MSE Mean: Approximately 64,458,433
MSE Standard Deviation: Approximately 572,737
The Random Forest model shows a significantly lower error compared to the Linear Regression model, indicating better predictive performance. The standard deviation of the MSE is also lower, suggesting more consistent results across different folds.

# Further Fine Tunning the Models

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from scipy.stats import boxcox
from datetime import datetime

import pandas as pd

# Load and preprocess the dataset
df = pd.read_csv('Downloads/vehicles.csv')



# Ensure 'price' is positive for Box-Cox transformation
df = df[df['price'] > 0]
df['price'], fitted_lambda = boxcox(df['price'])

# Define the outlier removal function
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply the outlier removal function
df = remove_outliers(df, 'price')
df = remove_outliers(df, 'odometer')

# Calculate 'car_age'
current_year = datetime.now().year
df['car_age'] = current_year - df['year']

# Handle NaN values for numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Check for and handle infinite values for numerical columns
if np.isinf(df[numerical_cols].values).any():
    df[numerical_cols] = df[numerical_cols].replace([np.inf, -np.inf], np.nan)
    df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Preprocessing
categorical_features = ['manufacturer', 'fuel', 'transmission']
numerical_features = ['year', 'odometer', 'car_age']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Define the target and features
X = df.drop(['price', 'id', 'region', 'model', 'condition', 'cylinders', 'title_status', 'VIN', 'drive', 'size', 'type', 'paint_color', 'state'], axis=1)
y = df['price']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





In [6]:
# Linear Regression with Grid Search
linear_model = LinearRegression()
linear_params = {'fit_intercept': [True, False]}
grid_search_linear = GridSearchCV(linear_model, linear_params, cv=5, scoring='neg_mean_squared_error')

linear_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('grid_search', grid_search_linear)])

linear_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['year', 'odometer',
                                                   'car_age']),
                                                 ('cat', OneHotEncoder(),
                                                  ['manufacturer', 'fuel',
                                                   'transmission'])])),
                ('grid_search',
                 GridSearchCV(cv=5, estimator=LinearRegression(),
                              param_grid={'fit_intercept': [True, False]},
                              scoring='neg_mean_squared_error'))])

In [6]:
#best parameters and scores
best_params_linear = grid_search_linear.best_params_
best_score_linear = -grid_search_linear.best_score_
#best_params_rf = grid_search_rf.best_params_
#best_score_rf = -grid_search_rf.best_score_

print("Best Params Linear Regression:", best_params_linear)
print("Best Score Linear Regression:", best_score_linear)
#print("Best Params Random Forest:", best_params_rf)
#print("Best Score Random Forest:", best_score_rf)

Best Params Linear Regression: {'fit_intercept': True}
Best Score Linear Regression: 14.501132827149544


In [8]:
from sklearn.metrics import mean_squared_error, r2_score


# Extract the ColumnTransformer from the pipeline
preprocessor = linear_pipeline.named_steps['preprocessor']

# Get the feature names after transformation
# For numerical features
num_features = preprocessor.transformers_[0][2]

# For categorical features
cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out()
cat_features = [str(i) for i in cat_features]  # Convert to string if necessary

# Combine all feature names
all_features = num_features + cat_features

# Extract the linear model from the pipeline
linear_model = linear_pipeline.named_steps['grid_search'].best_estimator_

# Get the coefficients
coefficients = linear_model.coef_

# Match coefficients to their corresponding feature names
feature_importance = pd.Series(coefficients, index=all_features)

print("Feature Importances:\n", feature_importance.sort_values(ascending=False))


# Predict on the test set
y_pred = linear_pipeline.predict(X_test)

# Evaluate the model performance
mse_test = mean_squared_error(y_test, y_pred)
r2_test = r2_score(y_test, y_pred)

print("Mean Squared Error on Test Set:", mse_test)
print("R-squared on Test Set:", r2_test)



Feature Importances:
 manufacturer_ferrari            9.979296
manufacturer_tesla              4.347453
fuel_diesel                     4.303727
manufacturer_porsche            3.961299
manufacturer_datsun             3.685299
manufacturer_aston-martin       3.604175
manufacturer_rover              2.367234
manufacturer_ram                2.220929
manufacturer_gmc                2.022157
manufacturer_lexus              1.379706
manufacturer_ford               1.016494
manufacturer_audi               1.004771
manufacturer_mercedes-benz      0.964282
manufacturer_land rover         0.922470
manufacturer_chevrolet          0.887187
manufacturer_jeep               0.737427
manufacturer_toyota             0.726311
transmission_other              0.591735
year                            0.574003
fuel_nan                        0.549466
manufacturer_bmw                0.495075
manufacturer_infiniti           0.459816
manufacturer_jaguar             0.424884
manufacturer_alfa-romeo         0.4

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

After thorough analysis and modeling, I have derived several key insights regarding the factors influencing used car prices. Here's a summary of my findings and recommendations based on the Linear Regression model:

Model Performance:
The model achieved an R-squared of approximately 0.4769 on the test set. This suggests that about 48% of the variability in used car prices is explained by the model, indicating a moderate level of predictive accuracy.
The Mean Squared Error (MSE) of 14.6605 suggests the model's predictions are reasonably close to the actual values, but there is room for improvement.
Influential Factors on Car Prices:
Brand Impact: 
Brands like Ferrari, Tesla, and Porsche significantly increase a car's price, indicating that brand value plays a crucial role in pricing.
Age and Mileage: The age and mileage of a car (as indicated by 'car_age' and 'odometer' readings) negatively affect the price, reflecting depreciation and wear.
Fuel Type Variations: Different fuel types impact prices differently, with diesel being positive and electric/hybrid having a negative influence.
Business Strategy Recommendations:
Inventory Focus: Prioritize acquiring and selling high-value brands that positively influence car prices. This can enhance profitability.
Pricing Strategy: Adjust pricing based on a car's age and mileage. Older and higher-mileage vehicles should be priced lower, reflecting their impact on value.
Market Trends: Stay informed about trends, especially regarding fuel types, as these preferences can shift over time and impact car values.
Potential for Further Analysis:
To increase the model's accuracy, exploring additional data sources or more detailed features could be valuable.
Advanced modeling techniques or additional feature engineering might uncover deeper insights, particularly for nuanced market segments.


### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

Executive Summary

Objective: To identify factors influencing used car prices, aiding the dealership in inventory and pricing decisions.

Key Findings:
Premium brands significantly increase car values.
Vehicle age and mileage inversely affect prices.
Fuel type variations play a notable role in pricing.

Recommendation: Focus on acquiring high-value brands and adjust pricing strategies based on age, mileage, and fuel type.

Data Overview

Dataset Description: The dataset includes data on 426,880 used cars, encompassing various makes, models, years, conditions, and other attributes.

Source: Compiled from a comprehensive online car marketplace.

Scope: Encompasses a diverse range of vehicles, offering a broad market perspective.

Methodology

Data Preparation:

Cleaning: Removed outliers and non-positive prices.

Transformation: Applied Box-Cox transformation for normalization.

Feature Engineering: Introduced 'car_age' as a new feature.

Model Selection: Chose Linear Regression for its interpretability and relevance to the data.

Hyperparameter Tuning: Employed GridSearchCV for optimal parameter selection.

Model Insights and Interpretation

Model Performance:

R-squared: 47.69%, indicating the model's moderate ability to predict price variations.

MSE: 14.6605, reflecting the average squared difference from actual prices.

Influential Factors:

Brands like Ferrari and Tesla boost prices.

Older cars and those with high mileage have lower prices.

Diesel fuel types positively influence prices, whereas electric/hybrid vehicles negatively impact prices.

Recommendations for Inventory Management

High-Value Brands: Prioritize acquiring and selling brands that have a strong positive impact on prices.

Pricing Strategy: Consider a car's age and mileage in pricing. Older and high-mileage cars should be priced lower.

Market Trend Adaptation: Stay updated on fuel type trends and consumer preferences.

Limitations and Future Work

Limitations:

The model explains only about 48% of the price variance.

Certain niche market segments might not be fully captured.

Future Directions:

Incorporating more granular data (specific models, detailed condition ratings).
Exploring non-linear models for potentially better predictions.

Conclusion

The analysis provides valuable information into price determinants in the used car market.
The dealership can take advantage of these findings to optimize inventory acquisition and develop pricing strategies.
